In [1]:
import pandas as pd

In [4]:
df = pd.read_csv('data/fetched_stm.csv', low_memory=False)
df.head()

,current_time,trip_id,route_id,start_date,stop_id,arrival_time,departure_time,schedule_relationship
0,1.745385e+09,285028348,189,20250422,54433,1745384718,1745384718,0
1,1.745385e+09,285028348,189,20250422,54444,1745384751,1745384751,0
2,1.745385e+09,285028348,189,20250422,54445,1745384785,1745384785,0
3,1.745385e+09,285028348,189,20250422,54451,1745384806,1745384806,0
4,1.745385e+09,285028348,189,20250422,54456,1745384829,1745384829,0


In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 250098 entries, 0 to 250097
Data columns (total 8 columns):
 #   Column                 Non-Null Count   Dtype  
---  ------                 --------------   -----  
 0   current_time           250098 non-null  float64
 1   trip_id                250098 non-null  int64  
 2   route_id               250098 non-null  object 
 3   start_date             250098 non-null  int64  
 4   stop_id                250098 non-null  int64  
 5   arrival_time           250098 non-null  int64  
 6   departure_time         250098 non-null  int64  
 7   schedule_relationship  250098 non-null  int64  
dtypes: float64(1), int64(6), object(1)
memory usage: 15.3+ MB


In [6]:
duplicate_mask = df.duplicated(subset=['trip_id', 'route_id', 'start_date', 'stop_id', 'arrival_time', 'departure_time', 'schedule_relationship'])
df[duplicate_mask]

,current_time,trip_id,route_id,start_date,stop_id,arrival_time,departure_time,schedule_relationship
13526,1.745388e+09,284778635,380,20250422,50495,0,0,1
13650,1.745388e+09,286573982,350,20250422,56686,0,0,1
13651,1.745388e+09,286573982,350,20250422,62367,0,0,1
13652,1.745388e+09,286573982,350,20250422,56582,0,0,1
13653,1.745388e+09,286573982,350,20250422,56578,0,0,1
...,...,...,...,...,...,...,...,...
250093,1.745414e+09,284726626,92,20250423,53892,1745413509,1745413509,0
250094,1.745414e+09,284726626,92,20250423,50531,1745413527,1745413527,0
250095,1.745414e+09,284726626,92,20250423,50507,1745413560,1745413560,0
250096,1.745414e+09,284726626,92,20250423,54362,1745413652,1745413652,0


In [7]:
df = df.drop_duplicates(subset=['trip_id', 'route_id', 'start_date', 'stop_id', 'arrival_time', 'departure_time', 'schedule_relationship'])

In [8]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 227570 entries, 0 to 250092
Data columns (total 8 columns):
 #   Column                 Non-Null Count   Dtype  
---  ------                 --------------   -----  
 0   current_time           227570 non-null  float64
 1   trip_id                227570 non-null  int64  
 2   route_id               227570 non-null  object 
 3   start_date             227570 non-null  int64  
 4   stop_id                227570 non-null  int64  
 5   arrival_time           227570 non-null  int64  
 6   departure_time         227570 non-null  int64  
 7   schedule_relationship  227570 non-null  int64  
dtypes: float64(1), int64(6), object(1)
memory usage: 15.6+ MB


In [10]:
df.to_csv('data/fetched_stm.csv', index=False)